In [ ]:
import numpy as np
import time
from numpy import ones,vstack
from numpy.linalg import lstsq

import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from random import random, randint

import os
import shutil

In [ ]:
def parse_trace(filename):
    with open(filename, "r") as f:
        lines = f.readlines()

    trajectory = []
    for i in range(0, len(lines), 2):
        line = lines[i]
        state = {}
        for tup in line.strip().split(" "):
            variable = tup.split("=")[0][1:-1]
            value = round(float(tup.split("=")[1]),3)
    #         print(variable, value)
            state[variable] = value

        trajectory += [state]
    return trajectory

In [ ]:
def generate_problem(h):
    
    pddl_str = "(define (problem problem_name) (:domain thermostat)\n"
    pddl_str += "\n"
    pddl_str += "(:init\n"
    pddl_str += "\t(= (x) 19)\n"
    pddl_str += "\t(off)\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\n"    
    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(>= (running_time) {h})\n".format(h=h)
    
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str
    

In [ ]:
def generate_observation(state, k):
    observation = (state[0],state[1],state[2])
    
    return observation

In [ ]:
def generate_decoding_problem(observations, error, time_precision):
    
    pddl_str = "(define (problem problem_name) (:domain decoding)\n"
    pddl_str += "\n"
    pddl_str += "(:objects\n"
    pddl_str += " ".join(["o{k}".format(k=i) for i in range(len(observations))]) + " - observation\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    pddl_str += "(:init\n"
    pddl_str += "\t(= (x) 19)\n"
    pddl_str += "\t(off)\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\n"
    pddl_str += "\t(= (time_precision) {time_precision})\n".format(time_precision=time_precision)
    pddl_str += "\t(= (x_error) {error})\n".format(error=error)
    for i in range(len(observations)):
        obs = observations[i]
        pddl_str +="\t(= (time_obs o{k}) {v})\n".format(k=i, v=obs[0])
        pddl_str +="\t(= (x_obs o{k}) {v})\n".format(k=i, v=obs[1])
        pddl_str += "\n"

    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(forall (?x - observation) (observed ?x))\n".format(h=h)
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str
    

In [ ]:
def generate_hydi(observations):
    
    hydi_str = "MODULE main\n"
    hydi_str += "VAR thermo : thermostat;\n"
    hydi_str += "VAR observation : observation(thermo.x, 0.05, 0.1);\n"
    hydi_str += "\n"

    hydi_str += "INVARSPEC !(observation.location = obs{});\n".format(str(len(observations)-1).zfill(2))
    hydi_str += "\n"

    hydi_str += "MODULE thermostat\n"
    hydi_str += "VAR\n"
    hydi_str += "\tlocation : {on, off};\n"
    hydi_str +="\tx : continuous;\n"
    hydi_str += "\n"

    hydi_str += "EVENT turn_on, turn_off;\n"
    hydi_str += "\n"

    hydi_str += "INIT\n"
    hydi_str += "\tlocation = off & x = 19;\n"
    hydi_str += "\n"

    hydi_str += "TRANS\n"
    hydi_str += "\tEVENT = turn_on ->\n"
    hydi_str += "\t\t(location = off & next(location) =  on & x < 19 & next(x) = x)\n"
    hydi_str += "\n"

    hydi_str += "TRANS\n"
    hydi_str += "\tEVENT = turn_off ->\n"
    hydi_str += "\t\t(location = on & next(location) =  off & x > 21 & next(x) = x)\n"
    hydi_str += "\n"

    hydi_str += "INVAR\n"
    hydi_str += "\t(location = off -> x >= 18) &\n"
    hydi_str += "\t (location = on -> x <= 22)\n"
    hydi_str += "\n"

    hydi_str += "FLOW\n"
    hydi_str += "\t(location = off -> der(x) = -0.2) &\n"
    hydi_str += "\t(location = on -> der(x) = 0.8)\n"
    hydi_str += "\n"

    hydi_str += "MODULE observation(x, x_error, t_precision)\n"
    hydi_str += "VAR\n"
    hydi_str += "\tlocation : {{{}}};\n".format(", ".join(["obs"+str(i) for i in range(len(observations))]))
    hydi_str += "\tt : continuous;\n"
    hydi_str += "\n"
    
    hydi_str += "EVENT {};\n".format(", ".join(["observe"+str(i) for i in range(1,len(observations))]))
    hydi_str += "INIT\n"
    hydi_str += "\tlocation = obs0 & t = 0;\n"
    hydi_str += "\n"

    for i in range(len(observations)-1):
        hydi_str += "TRANS\n"
        hydi_str += "\tEVENT = observe{} ->\n".format(i+1)
        hydi_str += "\t\t(location = obs{c} & next(location) =  obs{n} & x > {x} - x_error & x < {x} + x_error & t >= {t} - t_precision/2 & t < {t} + t_precision/2 & next(t) = t)\n".format(c=i, n=i+1, x=observations[i+1][1], t=observations[i+1][0])
    hydi_str += "\n"
    
    hydi_str += "FLOW\n"
    hydi_str += "\tder(t) = 1;"
    
    return hydi_str

In [ ]:
sample_rate = 1.0
num_instances = 1
horizons = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
errors = [0.05]
time_precision = [0.1]

In [ ]:
hfolder = "benchmarks/thermostat/hydi"
for pnum in range(num_instances):
    for h in horizons:
        pname = 'inst_{pnum}_{h}_5_100.hydi'.format(pnum=pnum, h=h)
        
        cmd = "hycomp -pre cpp -source {hfolder}/ic3.cmd {hfolder}/{pname}".format(hfolder=hfolder, pname=pname)
        t = time.time()
        os.system(cmd)
        elapsed = time.time() - t
        
        print(elapsed)